## In this TD, we aim at exploring the gradient descent method for optimizing smooth functions. 

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.signal as sig

## A. Gradient descent in 2D



In [ ]:
weights = np.array([0.1, 1])
f = lambda x : return np.sum(weights * x**2, axis=-1)
gradf = lambda x :weights * 2 * x

In [ ]:
def drawFunction(funct, minn, maxx, definition = 200, title = 'Title'):
  xx, yy = np.meshgrid(np.linspace(minn[0], maxx[0], definition), np.linspace(minn[1], maxx[1], definition))
  zz = funct(np.stack((xx, yy), axis=-1))
  axes = plt.gca()
  axes.set_ylim([minn[0], maxx[0]])
  axes.set_xlim([minn[1], maxx[1]])
  plt.contourf(xx, yy, zz, definition, cmap=plt.cm.rainbow)
  plt.axhline(0, color='black')
  plt.axvline(0, color='black')
  plt.title(title)

In [ ]:
plt.figure(figsize = (15,10))
drawFunction(f, [-1, -1], [1, 1], title='function')

### Ex: Implement a generic gradient descent algorithm taking an initial point, the gradient of the function, a number of steps and a stepsize $\varepsilon$.   Compute by hand  the Lipschitz constant of the gradient.

In [ ]:
# eps < 2/L
# where L = 2* max weights = 2 

def GradientDescent(x0, gradient, iterations=50, eps = 0.01):
    xValues = [x0]
    for _ in range(iterations-1):
        last = xValues[len(xValues)-1]
        xValues.append(last - eps * gradient(last))
    return xValues

### Ex:  Plot the gradient descent for different initializations and different step size of the gradient descent. Try to show that the convergence can be really slow when the ratio between the two weights is very large or very small. This ratio is often called the condition number. Other optimization algorithms have to be used in order to avoid this slow convergence such as quasi-Newton methods.

In [ ]:

ptsNb = 10
pts = np.random.rand(ptsNb, 2)*2-1

gradientPts = np.array(GradientDescent(pts, gradf, iterations=50, eps=0.05))

gradientPts = np.rollaxis(gradientPts, 1)

plt.figure(figsize = (15,15))

colors = cm.rainbow(np.linspace(0, 1, len(gradientPts)))

drawFunction(f, [-1, -1], [1, 1], title='function')

for x, c in zip(gradientPts, colors):
    plt.scatter(x[:, 0], x[:, 1], color=c, marker='x')

plt.show()

## B. Example on a physical system

### We consider a massive ball of mass $m$, subject to gravity and the force of the wind. The ball is at rest at time $0$ in position $x_0 \in \mathbb{R}^3$. We want to find the initial $v0$ such that the ball will land at a given final point $x_1$ at time $1$.

In [ ]:
# We implement the equations of the motion of the ball.
def Wind(t):
    return -np.asarray([-2,3,-0.2]) * np.cos(3*t) 

def GetPositionAtTime1(x0,v0,wind):
    number_of_timesteps = 1000
    timestep = 1.0/number_of_timesteps
    temp_x = x0.copy()
    temp_v = v0.copy()
    time = 0
    trajectory = [x0]
    times = [0]
    ## note that we use here a naive integrator: forward Euler.
    for i in range(number_of_timesteps):
        temp_x += timestep * (temp_v)
        temp_v += timestep * (2*np.asarray([0, 0, -1]) + wind(time)) #2 is a chosen value representing the gravity times the mass
        time += timestep
        trajectory.append(temp_x.copy())
        times.append(time)
    return temp_x,trajectory,times     

### We test the computation of the function

In [ ]:
# test 
x0 = np.asarray([0.,0.,0.]) # intial position
v0 = np.asarray([0.,0.,0.]) # initial speed
x1,trajectory,times = GetPositionAtTime1(x0,v0,Wind)
print("position at time 1",x1)
data = np.asarray(trajectory)

### We plot the trajectory.

In [ ]:
## prepare the trajectory
data = np.asarray(trajectory)
## Plot of the trajectory
from mpl_toolkits.mplot3d import Axes3D
mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
mpl.rcParams["figure.figsize"] = (20,20)

ax = fig.gca(projection='3d')

ax.plot(data[:,0], data[:,1], data[:,2], label='trajectory')
ax.legend()
ax.scatter(x0[0],x0[1],x0[2],"bo",linewidths = 20)
ax.scatter(x1[0],x1[1],x1[2],"ro",linewidths = 20)
plt.show()
print(x0,x1)

### Ex: Implement the following approximation of the gradient: $\partial_i f(x) \simeq \frac{f(x + \varepsilon e_i) - f(x)}{\varepsilon}$ with $e_i$ is the vector $(0,\ldots,1, \ldots , 0)$ where the $1$ is at the $i^{\text{th}}$ position. Then, recall that the gradient at point $x$ of the function $f$ is the vector $(\partial_i f(x))_{i \in 1,\ldots,n}$.

In [ ]:
## your code here.
def ApproximatedGradient(myFunction,x):
    return gradientAtx


### Ex: Using the previous approximation of the gradient, use a gradient descent with constant step size to solve the problem. For that, we define the $L^2$ penalty function between the target and the point position at time $1$ and we compute the gradient. Use it to solve the problem and plot the trajectory.

In [ ]:
def L2Penalty(finalPosition,target):
    return np.sum((finalPosition - target)**2)

def ComposedGradient(finalPosition,gradientFinalPosition,target):
    return 2*(finalPosition - t) * gradientFinalPosition

## your code here

## C. Example on a 2D image.

In [ ]:
import imageio as imio
colored_image = imio.imread('lena.png')
lena = np.sum(colored_image*[ 0.21, 0.72 ,0.07,0.0],axis=-1)
sub_defense = defense[::2,::2]
plt.gray()
plt.figure(figsize = (20,20))
plt.subplot(1,2,1)
plt.title("lena")
plt.imshow(defense)
plt.subplot(1,2,2)
plt.title("lena sub-sampled")
plt.imshow(sub_defense)

In [ ]:
def GenerateMask(shape,threshold = 2.0):
    temp = (np.random.rand(shape[0],shape[1])<0.5)*1.0
    filtering = np.array([[1,1],[1,1]])
    temp=  (sig.convolve(filtering,temp)<threshold)*1.0
    return temp[:shape[0],:shape[1]]

In [ ]:
mask = GenerateMask(np.shape(sub_defense),2.0)
print("averaged pixel value on the remaining pixels",np.sum(sub_defense*mask)/np.sum(mask))
print("percentage of remaining pixels",np.sum(mask)/np.size(mask))

plt.gray()
taille = 20
plt.figure(figsize = (taille,taille))
plt.subplot(1,2,1)
plt.title("lena",size = taille)
plt.imshow(mask);
plt.subplot(1,2,2);
plt.title("lena masked",size = taille)
plt.imshow(sub_defense*mask);

* ### Compute on a sheet of paper the gradient of the function $f: \mathbb{R}^d \mapsto \mathbb{R}$ defined by $f(x_1,\ldots,x_n) = \sum_{i = 1}^{n-1} \frac12| x_i - x_{i+1}|^2$. 
* ### Compute the Lipschitz constant of the gradient (remark that it is a linear function).
## We propose to reconstruct a candidate for our initial image as the minimizer of an energy functional $E(I) = \sum_{i = 1}^{\text{nbre lignes}} f(I[i,:])$ under the constraint that $I (1-\text{mask}) = \text{data}$.
* ### Copy your gradient descent code and modify it to implement the projected gradient descent algorithm.
* ### Apply the algorithm to the data and comment. How do you suggest to modify the energy functional in order to improve the results ?

In [ ]:
# your code here.

### Answer: your answer here. $##############$